In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
future_cols = ['date', 'time', 'f_open', 'f_high', 'f_low', 'f_close', 'f_vol']
kospi200_cols = ['date', 'time', 'k200_open', 'k200_high', 'k200_low', 'k200_close', 'k200_vol']
kospi_cols = ['date', 'time', 'k_open', 'k_high', 'k_low', 'k_close', 'k_vol']
kosdaq_cols = ['date', 'time', 'd_open', 'd_high', 'd_low', 'd_close', 'd_vol']

In [3]:
# futures
# 2020/07/17, 9:00:00
# 2020/11/09, 15:45:00

# kospi200, kospi, kosdaq
# 2020/07/16, 15:30:00
# 2020/11/09, 15:30:00

In [4]:
def convert_csv():
    dataF = pd.read_excel('data-files/futures.xlsx')
    dataF.columns = future_cols
    dataF.set_index(['date', 'time'], inplace=True)
    dataF.sort_index(inplace=True)
    dataF = dataF.loc[('2020/07/17', '09:00:00'):('2020/11/09', '15:45:00'), :]
    dataF.to_csv('data-files/futures.csv', index=True, encoding='utf-8-sig')
    dataF.info()

    dataK200 = pd.read_excel('data-files/kospi200.xlsx')
    dataK200.columns = kospi200_cols
    dataK200.set_index(['date', 'time'], inplace=True)
    dataK200.sort_index(inplace=True)
    dataK200 = dataK200.loc[('2020/07/16', '15:30:00'):('2020/11/09', '15:30:00'), :]
    dataK200.info()

    dataK = pd.read_excel('data-files/kospi.xlsx')
    dataK.columns = kospi_cols
    dataK.set_index(['date', 'time'], inplace=True)
    dataK.sort_index(inplace=True)
    dataK = dataK.loc[('2020/07/16', '15:30:00'):('2020/11/09', '15:30:00'), :]
    dataK.info()

    dataD = pd.read_excel('data-files/kosdaq.xlsx')
    dataD.columns = kosdaq_cols
    dataD.set_index(['date', 'time'],inplace=True)
    dataD.sort_index(inplace=True)
    dataD = dataD.loc[('2020/07/16', '15:30:00'):('2020/11/09', '15:30:00'), :]
    dataD.info()

    dataK200K = dataK200.merge(dataK, how='left', left_index=True, right_index=True)
    dataK200KD = dataK200K.merge(dataD, how='left', left_index=True, right_index=True)
    dataK200KD.to_csv('data-files/k200kd.csv', index=True, encoding='utf-8-sig')
    dataK200KD.info()
    
# convert_csv()

In [10]:
def make_allD():
    furtures = pd.read_csv('data-files/futures.csv')
    k200kd = pd.read_csv('data-files/k200kd.csv')

    dataF = furtures.set_index(['date', 'time'])
    dataK = k200kd.set_index(['date', 'time'])

    dataFK = dataF.merge(dataK, how='outer', left_index=True, right_index=True)
    dataFK.to_csv('data-files/allD.csv', index=True, encoding='utf-8-sig')

    dataFK.info()

# make_allD()

In [9]:
# ('2020/07/16', '15:30:00'),
# ('2020/09/10', '15:20:00'),
# ('2020/09/10', '15:30:00')
dataFK = pd.read_csv('data-files/allD.csv')

null = dataFK.fillna(-1.0)
# print(null.loc[null['f_open'] == -1.0].index)
nullD = null.loc[null['k_open'] == -1.0]
nullD.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141 entries, 1 to 30478
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1141 non-null   object 
 1   time        1141 non-null   object 
 2   f_open      1141 non-null   float64
 3   f_high      1141 non-null   float64
 4   f_low       1141 non-null   float64
 5   f_close     1141 non-null   float64
 6   f_vol       1141 non-null   float64
 7   k200_open   1141 non-null   float64
 8   k200_high   1141 non-null   float64
 9   k200_low    1141 non-null   float64
 10  k200_close  1141 non-null   float64
 11  k200_vol    1141 non-null   float64
 12  k_open      1141 non-null   float64
 13  k_high      1141 non-null   float64
 14  k_low       1141 non-null   float64
 15  k_close     1141 non-null   float64
 16  k_vol       1141 non-null   float64
 17  d_open      1141 non-null   float64
 18  d_high      1141 non-null   float64
 19  d_low       1141 non-null 

In [9]:
# result = testFK[['date', 'k_open', 'k_high', 'k_low', 'k_close', 'k_vol']].groupby(by=['date']).count()
# date_list = sorted(np.unique(dataFK['date']))
# date_list
# np.unique(dataFK['time'])

In [13]:
k200_nulls = ['k200_open', 'k200_high', 'k200_low', 'k200_close', 'k200_vol']
k_nulls = ['k_open', 'k_high', 'k_low', 'k_close', 'k_vol']
d_nulls = ['d_open', 'd_high', 'd_low', 'd_close', 'd_vol']

def fill_values(df, origin_id, copy_id):

    close = df.loc[origin_id, 'k200_close']
    df.loc[copy_id, k200_nulls] = [close, close, close, close, 0]

    close = df.loc[origin_id, 'k_close']
    df.loc[copy_id, k_nulls] = [close, close, close, close, 0]

    close = df.loc[origin_id, 'd_close']
    df.loc[copy_id, d_nulls] = [close, close, close, close, 0]


In [18]:
def make_futuresD():

    dataFK = pd.read_csv('data-files/allD.csv')
    dataFK2 = dataFK.set_index(['date', 'time'])

    date_list = sorted(np.unique(dataFK['date']))

    null_time1 = ('15:30:00', '09:00:00') # 전날, 오늘
    null_time2 = ('15:30:00', ['15:31:00', '15:32:00', '15:33:00', '15:34:00', '15:45:00']) # 오늘, 오늘
    null_time3 = ('15:20:00', ['15:21:00', '15:22:00', '15:23:00', '15:24:00', '15:25:00', #오늘, 오늘
                            '15:26:00', '15:27:00', '15:28:00', '15:29:00'])

    # 0: 2020/07/16 ~
    for idx, da in enumerate(date_list):

        if idx <= 0:
            continue

        origin_id = (date_list[idx - 1], null_time1[0])
        copy_id = (da, null_time1[1])
        fill_values(dataFK2, origin_id, copy_id)
        
        if da == '2020/09/10':
            continue

        origin_id = (da, null_time2[0])
        
        for ti2 in null_time2[1]:
            copy_id = (da, ti2)
            fill_values(dataFK2, origin_id, copy_id)
            
        
        origin_id = (da, null_time3[0])

        for ti3 in null_time3[1]:
            copy_id = (da, ti3)
            fill_values(dataFK2, origin_id, copy_id)

    dataFK2.dropna(axis=0, inplace=True)
    dataFK2.to_csv('data-files/futuresD.csv', index=True, encoding='utf-8-sig')

    dataFK2.info()    

# make_futuresD()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 30476 entries, ('2020/07/17', '09:00:00') to ('2020/11/09', '15:45:00')
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   f_open      30476 non-null  float64
 1   f_high      30476 non-null  float64
 2   f_low       30476 non-null  float64
 3   f_close     30476 non-null  float64
 4   f_vol       30476 non-null  float64
 5   k200_open   30476 non-null  float64
 6   k200_high   30476 non-null  float64
 7   k200_low    30476 non-null  float64
 8   k200_close  30476 non-null  float64
 9   k200_vol    30476 non-null  float64
 10  k_open      30476 non-null  float64
 11  k_high      30476 non-null  float64
 12  k_low       30476 non-null  float64
 13  k_close     30476 non-null  float64
 14  k_vol       30476 non-null  float64
 15  d_open      30476 non-null  float64
 16  d_high      30476 non-null  float64
 17  d_low       30476 non-null  float64
 18  d_close     30476 no

In [19]:
dataF = pd.read_csv('data-files/futuresD.csv')
dataF.info()
dataF.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30476 entries, 0 to 30475
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        30476 non-null  object 
 1   time        30476 non-null  object 
 2   f_open      30476 non-null  float64
 3   f_high      30476 non-null  float64
 4   f_low       30476 non-null  float64
 5   f_close     30476 non-null  float64
 6   f_vol       30476 non-null  float64
 7   k200_open   30476 non-null  float64
 8   k200_high   30476 non-null  float64
 9   k200_low    30476 non-null  float64
 10  k200_close  30476 non-null  float64
 11  k200_vol    30476 non-null  float64
 12  k_open      30476 non-null  float64
 13  k_high      30476 non-null  float64
 14  k_low       30476 non-null  float64
 15  k_close     30476 non-null  float64
 16  k_vol       30476 non-null  float64
 17  d_open      30476 non-null  float64
 18  d_high      30476 non-null  float64
 19  d_low       30476 non-nul

,date,time,f_open,f_high,f_low,f_close,f_vol,k200_open,k200_high,k200_low,...,k_open,k_high,k_low,k_close,k_vol,d_open,d_high,d_low,d_close,d_vol
0,2020/07/17,09:00:00,289.65,290.10,289.45,290.10,2952.0,289.25,289.25,289.25,...,2183.76,2183.76,2183.76,2183.76,0.0,775.07,775.07,775.07,775.07,0.0
1,2020/07/17,09:01:00,290.05,290.90,290.05,290.85,3622.0,289.90,290.63,289.90,...,2187.85,2191.66,2187.85,2191.58,15381.0,776.97,777.51,776.96,777.39,33111.0
2,2020/07/17,09:02:00,290.85,291.00,290.60,290.80,2421.0,290.65,290.97,290.37,...,2193.37,2194.98,2193.37,2193.37,7324.0,777.48,777.90,777.48,777.72,14170.0
3,2020/07/17,09:03:00,290.80,291.15,290.80,291.10,1855.0,290.37,291.02,290.37,...,2192.47,2195.46,2192.47,2194.76,5540.0,777.70,777.86,777.65,777.77,13601.0
4,2020/07/17,09:04:00,291.15,291.30,291.00,291.00,2633.0,291.07,291.23,290.90,...,2196.46,2197.28,2196.36,2196.63,8572.0,777.79,778.18,777.76,778.18,11594.0
5,2020/07/17,09:05:00,291.00,291.25,290.95,291.25,1427.0,291.15,291.19,290.88,...,2196.97,2196.97,2195.50,2196.17,5507.0,778.10,778.23,778.07,778.23,9365.0
6,2020/07/17,09:06:00,291.25,291.30,290.95,291.00,1467.0,291.20,291.23,290.98,...,2197.35,2197.35,2196.13,2196.64,5194.0,778.24,778.24,778.03,778.03,10205.0
7,2020/07/17,09:07:00,291.00,291.20,291.00,291.20,1031.0,290.97,291.20,290.97,...,2195.55,2196.60,2195.55,2196.60,4432.0,778.08,778.22,778.05,778.20,10233.0
8,2020/07/17,09:08:00,291.15,291.25,291.10,291.25,878.0,291.10,291.22,290.95,...,2195.81,2196.61,2195.27,2195.53,4569.0,778.29,778.41,778.27,778.40,9197.0
9,2020/07/17,09:09:00,291.25,291.40,291.15,291.35,1231.0,291.21,291.34,290.99,...,2196.61,2196.61,2195.63,2196.41,4567.0,778.25,778.25,777.83,777.83,9124.0
